<a href="https://colab.research.google.com/github/stsan9/EndoMondoResearchERSP/blob/master/Data%20Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import json
import numpy as np
import math
from datetime import datetime

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
properPath = '/content/gdrive/My Drive/EndoMondoData/endomondoHR_proper.json'

In [0]:
data = []
with open(properPath) as f:
    for l in f:
        data.append(eval(l))

In [0]:
dataframe = pd.DataFrame.from_dict(data)

In [0]:
dataframe['average_altitude'] = dataframe['altitude'].apply(np.mean)
dataframe['time_start'] = dataframe['timestamp'].apply(lambda x: x[0])
dataframe['average_heart_rate'] = dataframe['heart_rate'].apply(np.mean)
dataframe.drop(columns=['speed', 'url', 'altitude'], inplace=True)

In [0]:
dataframe = pd.concat([dataframe,pd.get_dummies(dataframe['gender'], prefix='gender')],axis=1)
dataframe.drop(['gender'],axis=1, inplace=True)

In [0]:
lat_list = dataframe['latitude']
long_list = dataframe['longitude']

dist_list = []
time_list = []

for i in range(dataframe.shape[0]):
  dist = 0
  time_list.append(dataframe['timestamp'][i][-1] - dataframe['timestamp'][i][0])
  for j in range(len(dataframe['latitude'][i]) -1):
    lat1 = math.radians(lat_list[i][j])
    lon1 = math.radians(long_list[i][j])
    lat2 = math.radians(lat_list[i][j + 1])
    lon2 = math.radians(long_list[i][j + 1])

    R = 6373.0

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    dist += R * c
  dist_list.append(dist)

In [0]:
def time_scale(num):
  return num/60/60

In [0]:
dataframe['derived_distance (km)'] = dist_list
dataframe['dervided_speed (km/h)'] = dataframe['derived_distance (km)'] / pd.Series(time_list).apply(time_scale)

In [0]:
sport_key = list(dataframe['sport'].unique())

def sport_convert(sport):
  for i in range(len(sport_key)):
    if sport_key[i] == sport:
      return i+1

dataframe['sport'] = dataframe['sport'].apply(sport_convert)

In [0]:
all_users=dataframe["userId"].unique()
saved_stamps = list(dataframe['time_start'])
times = []
for user in all_users:
  user_data = dataframe.loc[dataframe["userId"] == user].sort_values("time_start")
  arr = np.diff(user_data["time_start"])
  arr=np.append(arr,[0])
  user_data["time_start"]=arr.tolist()
  indexNames = dataframe[dataframe["userId"] == user].index
  dataframe.drop(indexNames,inplace=True)
  dataframe = dataframe.append(user_data,ignore_index = True)

dataframe = dataframe.rename( columns= { "time_start" : "hrs_to_next" } )
dataframe['time_start'] = saved_stamps
dataframe['hrs_to_next'] = dataframe['hrs_to_next'].apply(time_scale)

In [0]:
df = dataframe.copy()
df = df[df['hrs_to_next'] != 0]
median_gap = np.median(df['hrs_to_next'])
print(median_gap/24)

1.9372280092592593


In [0]:
#Converting hrs_to_next to month ranges
def hrs_to_month(num):
  return num/24/30
dataframe["months_to_next"]=dataframe["hrs_to_next"].apply(hrs_to_month)

In [0]:
#Converting time_start to number of month
def extract_month(num):
  date=datetime.fromtimestamp(num)
  return date.month
dataframe["month"]=dataframe["time_start"].apply(extract_month)

In [0]:
dataframe = df.sort_values('time_start', ascending=False).reset_index(drop=True)
dataframe.head()

,longitude,latitude,sport,id,heart_rate,timestamp,userId,average_altitude,hrs_to_next,average_heart_rate,gender_female,gender_male,gender_unknown,derived_distance (km),dervided_speed (km/h),time_start
0,"[18.3869108, 18.3868904, 18.3865188, 18.385767...","[54.71762, 54.7176174, 54.7175796, 54.7174957,...",1,472806542,"[83, 84, 89, 94, 98, 98, 101, 100, 107, 110, 1...","[1223070209, 1223070210, 1223070218, 122307022...",653747,32.822155,72.064722,107.978,0,1,0,30.013173,23.524368,1554623190
1,"[128.29841653816402, 128.2981062401086, 128.29...","[36.23876868747175, 36.23854053206742, 36.2387...",1,610057267,"[108, 103, 103, 104, 106, 111, 121, 123, 133, ...","[1443534904, 1443534929, 1443534945, 144353496...",11002376,96.758000,89.077778,140.666,0,1,0,37.319676,12.836884,1550233552
2,"[127.32873830012977, 127.32873444445431, 127.3...","[37.54560286179185, 37.54556933417916, 37.5453...",1,612019809,"[80, 99, 108, 115, 108, 106, 118, 127, 126, 12...","[1443855584, 1443855620, 1443855639, 144385565...",11002376,110.415600,29547.230833,155.180,0,1,0,31.841821,19.887327,1550225615
3,"[18.642657, 18.642637, 18.642661, 18.642731, 1...","[-33.854448, -33.85428, -33.853918, -33.85349,...",1,32971195,"[108, 110, 113, 114, 109, 104, 97, 102, 111, 1...","[1208975271, 1208975278, 1208975286, 120897529...",28064,164.366084,47.340556,146.902,0,1,0,27.571008,24.926074,1451658916
4,"[28.2229102, 28.2229031, 28.2229155, 28.222928...","[-25.7221224, -25.7221948, -25.7223643, -25.72...",3,101087026,"[83, 84, 100, 115, 125, 129, 131, 132, 136, 13...","[1350559938, 1350559942, 1350559947, 135055995...",860533,1294.496800,23.987500,164.566,0,1,0,11.740080,11.547620,1451653209


In [0]:
a = dataframe.groupby('userId').count()
good_users = list(a[a['id'] > 20].index)
dataframe = dataframe[dataframe['userId'].isin(good_users)]
dataframe['hrs_to_next2'] = dataframe['hrs_to_next'].copy()
dataframe["months_to_next"]=dataframe["hrs_to_next"].apply(hrs_to_month)

In [0]:

utt1 = dataframe.groupby('userId').agg(
    {'hrs_to_next': np.median,
     'time_start': lambda x: x[x >= (x.iloc[1] - median_gap*60*60)].count()-2,
     'sport': lambda x: x.iloc[1],
     'hrs_to_next2': lambda x: int(x.iloc[1] <= median_gap),
     'average_heart_rate': lambda x: x.iloc[1],
     'dervided_speed (km/h)': lambda x: x.iloc[1],
     'derived_distance (km)': lambda x: x.iloc[1],
     'average_altitude': lambda x: x.iloc[1],
     'months_to_next': lambda x: x.iloc[2],
     'gender_female': lambda x: x.iloc[1],
     'gender_male': lambda x: x.iloc[1],
     'gender_unknown': lambda x: x.iloc[1]
     }
    )
utt2 = dataframe.groupby('userId').agg(
    {'hrs_to_next': lambda x: x.iloc[2],
     'hrs_to_next2': lambda x: x.iloc[3],
     'months_to_next': lambda x: x.iloc[3]
     }
     )

In [0]:
utt1.columns = [
                'user_median', 'num_recent_workouts', 'recent_sport', 
                'workout_soon', 'recent_heart_rate', 'recent_speed',
                'recent_distance', 'recent_altitude', 'months_to_next1',
                'gender_female', 'gender_male', 'gender_unknown'
                ]
utt2.columns = [
                'hrs_to_next1', 'hrs_to_next2', 'months_to_next2'
                ]

In [0]:
new_user_time_table = utt1.merge(utt2, left_index=True, right_index=True)
new_user_time_table.to_csv(r'/content/gdrive/My Drive/Group Research Materials/McAuley/NewUserTimeTable.csv')